In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time
import pandas as pd
import re

In [3]:
# Cria dataframe para receber o conteúdo
Campos = ['Data',
           'Alagamento',
           'Região',
           'Bairro',
           'Tipo',
           'Horário Inicial',
           'Horário Final',
           'Endereço',
           'Sentido',
           'Referência'
           ]
df = pd.DataFrame(columns=Campos)

In [6]:
# Montagem dinâmica das datas a serem varridas:
start = date(2023, 1, 1)
end = date(2023, 1, 5)
# Quando a função estiver totalmente funcional
# end = date.today()
data_pesquisada = start
dia = timedelta(days=1)

In [ ]:
# Loop para baixar vários dias. Ainda não está pronto
while data_pesquisada < end:
    data_formatada = data_pesquisada.strftime('%m')+'%2F'+data_pesquisada.strftime('%d')+'%2F'+data_pesquisada.strftime('%Y')
    URL_pesquisada = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca='+data_formatada+'&enviaBusca=Buscar'
#    print(URL_pesquisada)
    
    # Faz a requisição para a página
    response = requests.get(URL_pesquisada)
    
    # Analisa o HTML da página com o BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    print('Data pesquisada:',data_pesquisada)
    for el in soup.find_all('div',{"class": 'yui3-u col-alagamentos'}):
        print(el.get_text())
    
    data_pesquisada = data_pesquisada + dia
    time.sleep(1)

In [7]:
URL = 'https://www.cgesp.org/v3/alagamentos.jsp?dataBusca=04%2F01%2F2023&enviaBusca=Buscar'
# Faz a requisição para a página
response = requests.get(URL)
    
# Analisa o HTML da página com o BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [13]:
index = 0
alagamento = 1

for tabelas in soup.findAll('div',{'class':'ponto-de-alagamento'}):
    # Primeiro, vamos achar a quantidade de pontos de alagamento
    print(tabelas)
    string = soup.find('td',{'class':'total-pts arial-bairros-alag'}).get_text()
    no_alagamentos = int(string[0:string.find('pt')-1])
    print('O número de alagamentos encontrados foi: ',no_alagamentos)

    while alagamento < no_alagamentos+1:
        # Data: a mais fácil. É o dia que está sendo iterado
        df.at[index,'Data'] =  data_pesquisada

        # Alagamento: este campo é para indicar que se choveu ou não nesse dia
        df.at[index,'Alagamento'] = 1

        #'Região'
        df.at[index,'Região'] = tabelas.find('h1',{'class':'tit-bairros'}).get_text()

        #'Bairro',
        df.at[index,'Bairro'] = tabelas.find('td',{'class':'bairro arial-bairros-alag linha-pontilhada'}).get_text(strip=True)

        #'Tipo',
        filho = tabelas.find('div',{'class':'ponto-de-alagamento'})
        filhos = filho.findChildren()
        df.at[index,'Tipo'] = filhos[1]['title']

        #'Horário Inicial',
        texto=filhos[3].text
        horas =re.findall('(?:(\d\d?):)?([0-5][0-9])',texto)
        df.at[index,'Horário Inicial'] = str(horas[0][0])+':'+str(horas[0][1])

        #'Horário Final'
        df.at[index,'Horário Final'] = str(horas[1][0])+':'+str(horas[1][1])

        # Endereço
        df.at[index,'Endereço'] = texto[texto.find(horas[1][1])+2:len(texto)]

        #'Sentido',
        texto = filhos[6].text
        info = texto.find('Ref')
        df.at[index,'Sentido'] = texto[8:texto.find('Ref')]

        #'Referência'
        df.at[index,'Referência'] = texto[texto.find('Ref')+11:len(texto)]

        index = index + 1
        alagamento = alagamento + 1


<div class="ponto-de-alagamento">
<ul>
<li class="inativo-intransitavel" title="Inativo Intransitável"></li>
<li class="espaco"></li>
<li class="arial-descr-alag col-local">De 16:49 a 17:03<br/>R MANOEL DUTRA</li>
<li class="ln-vert"></li>
<li class="arial-descr-alag">Sentido: AMBOS<br/>Referência: PRAÇA 14 BIS</li>
</ul>
</div>
O número de alagamentos encontrados foi:  2


AttributeError: 'NoneType' object has no attribute 'get_text'

In [9]:
print(df)

         Data Alagamento  Região Bairro                   Tipo  \
0  2023-01-01          1  Centro     Sé  Inativo Intransitável   
1  2023-01-01          1  Centro     Sé  Inativo Intransitável   

  Horário Inicial Horário Final        Endereço Sentido     Referência  
0           16:49         17:03  R MANOEL DUTRA   AMBOS   PRAÇA 14 BIS  
1           16:49         17:03  R MANOEL DUTRA   AMBOS   PRAÇA 14 BIS  


In [ ]:
string = soup.find('td',{'class':'total-pts arial-bairros-alag'}).get_text()

In [ ]:
filho = soup.findAll('div',{'class':'ponto-de-alagamento'})
filho